In [4]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [5]:
import json
from glob import glob
from pathlib import Path
from typing import Iterator, NewType, Union
from datetime import datetime
from warnings import warn
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame

from sppp.schema import FeatureCollection

idx: slice = pd.IndexSlice

ModuleNotFoundError: No module named 'sppp'

In [ ]:
def iterpaths() -> Iterator[Path]:
    yield from (Path(file) for file in glob("/workspaces/sppp/data/new/*.json"))


def to_midf() -> pd.DataFrame:
    def generate():
        for path in iterpaths():
            with path.open("rb") as f:
                try:
                    fc: FeatureCollection = json.load(f)
                except json.JSONDecodeError:
                    warn(f"json error decoding {path}" )
                    continue
                df: GeoDataFrame = GeoDataFrame.from_features(fc["features"])
                df["validTime"] = datetime.strptime(fc["validTime"], "%Y%m%d_%H%M%S %Z")
                yield df

    return pd.concat(generate()).set_index(["validTime", "ID"])


df = to_midf()
# df

In [ ]:
#
df.loc[df["PS"].astype(float) > 90, "PS"]
# df.loc[idx[:, "89519"], :]
import pyarrow

NameError: name 'df' is not defined